In [11]:
import os
import pandas as pd
import numpy as np
import pathlib
import xarray as xr
import rioxarray

In [12]:
read_file_path: str = r"./../data_global_surge/1106_data/field_2024-11-10_12_00_00.f0.nc"
"""读取文件"""
data = xr.open_dataset(read_file_path)
data

<xarray.Dataset>
Dimensions:    (Time: 1, latitude: 961, longitude: 2201)
Coordinates:
  * longitude  (longitude) float64 100.0 100.0 100.1 100.2 ... 209.9 209.9 210.0
  * latitude   (latitude) float64 -45.0 -44.95 -44.9 -44.85 ... 2.9 2.95 3.0
  * Time       (Time) datetime64[ns] 2024-11-10T12:00:00
Data variables:
    h          (Time, latitude, longitude) float32 ...
    maskLand   (latitude, longitude) float64 ...
    hMax       (latitude, longitude) float32 ...

In [13]:
var_name='hMax'

In [14]:

# TODO:[-] 24-09-23 注意需要基于 maskland为掩码文件，将h与hmax陆地部分掩码掉
data[var_name]=data[var_name].where(data['maskLand']==0,np.nan)
# data['hMax']=data['hMax'].where(data['maskLand']==0,np.nan)
data

<xarray.Dataset>
Dimensions:    (Time: 1, latitude: 961, longitude: 2201)
Coordinates:
  * longitude  (longitude) float64 100.0 100.0 100.1 100.2 ... 209.9 209.9 210.0
  * latitude   (latitude) float64 -45.0 -44.95 -44.9 -44.85 ... 2.9 2.95 3.0
  * Time       (Time) datetime64[ns] 2024-11-10T12:00:00
Data variables:
    h          (Time, latitude, longitude) float32 ...
    maskLand   (latitude, longitude) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    hMax       (latitude, longitude) float32 nan nan nan ... 0.2284 0.2289

In [15]:
# 掩码时，需要将超出阈值的部分也进行掩码
# TypeError: ufunc 'bitwise_or' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
data_mask=(data[var_name]<=-0.2)|(data[var_name]>=0.2)

In [5]:
data_mask

<xarray.DataArray 'h' (Time: 1, latitude: 961, longitude: 2201)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Coordinates:
  * longitude  (longitude) float64 100.0 100.0 100.1 100.2 ... 209.9 209.9 210.0
  * latitude   (latitude) float64 -45.0 -44.95 -44.9 -44.85 ... 2.9 2.95 3.0
  * Time       (Time) datetime64[ns] 2024-11-10T12:00:00

In [16]:
filtered_ds_h = data.where(data_mask, drop=False)

In [17]:
data_standard=filtered_ds_h.sortby('latitude',ascending=False)

In [18]:
first_time = data_standard["Time"].values[0]

In [19]:
first_dr = data_standard.sel(Time=first_time)[var_name]
# 设置空间坐标系与分辨率
first_dr.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
first_dr.rio.write_crs("EPSG:4326", inplace=True)

<xarray.DataArray 'hMax' (latitude: 961, longitude: 2201)>
array([[       nan,        nan,        nan, ..., 0.22810812, 0.22836801,
        0.22893688],
       [       nan,        nan,        nan, ..., 0.22645205, 0.22644179,
        0.22625066],
       [       nan,        nan,        nan, ..., 0.22492991, 0.22482258,
        0.22482258],
       ...,
       [       nan,        nan,        nan, ..., 0.30864218, 0.30855498,
        0.30828902],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
  * longitude    (longitude) float64 100.0 100.0 100.1 ... 209.9 209.9 210.0
  * latitude     (latitude) float64 3.0 2.95 2.9 2.85 ... -44.9 -44.95 -45.0
    Time         datetime64[ns] 2024-11-10T12:00:00
    spatial_ref  int32 0
Attributes:
    units:         m
    long_name:     max storm surge
    grid_mapping:  spatial_ref

In [21]:
out_put_file_path: str = (
    r"./../data_global_surge/1106_data/field_2024-11-03_16_00_00.f0_hmax.tif"
)
# first_dr.rio.to_raster(out_put_file_path,diver='GTiff',compress='deflate')
first_dr.rio.to_raster(out_put_file_path,diver='GTiff',compress='DEFLATE')